# Instance-Based Authorship Attribution

## <font color='blue'>importing necessary libraries </font>

In [1]:
import io, os, sys, types
import IPython
%run stylometric_Features.ipynb import FeatureExtration
import pandas as pd
import seaborn as sns
import nltk
import glob
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings("ignore")

from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from scipy import sparse
import math
from sklearn.preprocessing import normalize
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
import operator
import itertools
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB

In [2]:
#from stylometric_Features import LexicalFeatures,SyntacticFeatures
#from stylometric_Features import FeatureExtration
#from ngrams_Features import train_model

#data reading mutilple files
#hrudayashiva-instance-->01-12
#ravi beligeri-isntance-->13-25
#somashaker-isntance-->26-36

## reading file and extracting Lexical Features

In [3]:
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
all_instances={
'hrudayashiva':["01","02","03","04","05","06","07","08","09",10,11,12],
    'ravibeliger':[13,14,15,16,17,18,19,20,21,22,23,24,25],
    'somashaker':[26,27,28,29,30,31,32,33,34,35,36]
}

stop_words_list = "C://Users/RAVIKUMAR/PycharmProjects/Authorship_Attribution_Instance/Data_2/new_stop_words.txt"

data_folder = "C://Users/RAVIKUMAR/PycharmProjects/Authorship_Attribution_Instance/Data_2/train"
instance_by_author={}
for author,AllFiles_Per_author in all_instances.items():
    for i in AllFiles_Per_author:
        for name in glob.glob(f"C://Users/RAVIKUMAR/PycharmProjects/Authorship_Attribution_Instance/Data_2/train/instance{i}.txt"):
            with open(name, encoding='utf-8') as file:
                instance_by_author[author+str(i)]=file.read()
all_authors=[]
text=[]
original = pd.DataFrame()
for author, file in instance_by_author.items():
    all_authors.append(author)
    text.append(file)
original["Authors with id"] = all_authors
original["text files"]=text
df=pd.DataFrame()  

In [4]:
#original

In [5]:
# labelencoder = LabelEncoder()
# y = labelencoder.fit_transform(y)
X=original["text files"]
y=all_authors
# 80-20 splitting the dataset (80%->Training and 20%->Validation)
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=1234)

In [6]:
print("------------------LEXICAL FEATURES EXTRACTING----------")
print()
author_fvs={}
feature_vectors=[]
for (text,author) in zip(X_train, y_train):
    author_fvs[author]=FeatureExtration(text)
    #feature_vectors.append(FeatureExtration(text))
print("successfully extracted all the features\n \n ")
print("--------LEXICAL FEATURES EXTRACTED AND STORED IN VECTOR------")

------------------LEXICAL FEATURES EXTRACTING----------

successfully extracted all the features
 
 
--------LEXICAL FEATURES EXTRACTED AND STORED IN VECTOR------


In [7]:
train_data = pd.DataFrame()
train_data["X_train"] =X_train
train_data["y_train"] = y_train
train_data["features"] =author_fvs.values()

In [8]:
#train_data.head(5)

In [9]:
Y = pd.Series(train_data["y_train"])
Y.replace(r'(^ra.*)','ravibeligeri',regex=True, inplace = True)
Y.replace(r'(^hr.*)','hrudayashiva',regex=True, inplace = True)
Y.replace(r'(^som.*)','somashaker',regex=True, inplace = True)
train_data["y_train"] = Y
y_train = train_data["y_train"]
vectors = train_data["features"]
# labelencoder = LabelEncoder()
# y = labelencoder.fit_transform(y)
arr = [0,0,0,0,0,0]
for i in vectors:
    for k in range(500-len(i)):
        i.append(arr)
arr = (np.array(vectors))

In [10]:
# mean normalization of the data . converting into normal distribution having mean=0 , -0.1<x<0.1
#sc = StandardScaler().fit(arr)

In [11]:
def flatten_list(_2d_list):
    flat_list = []
    # Iterate through the outer list
    for element in _2d_list:
        if type(element) is list:
            # If the element is of type list, iterate through the sublist
            for item in element:
                flat_list.append(item)
        else:
            flat_list.append(element)
    return flat_list
flatten_final=[]
for i in range(len(vectors)):
    flatten_final.append(flatten_list(arr[i]))

# nested_list = vectors[0]
# print('Original List', len(nested_list)
print('Transformed Flat List', len(flatten_final))

Transformed Flat List 28


In [12]:
flatten_final = np.asarray(flatten_final)
a = flatten_final
col_mean = np.nanmean(a, axis=0)
#Find indices that you need to replace
inds = np.where(np.isnan(a))
#Place column means in the indices. Align the arrays using take
a[inds] = np.take(col_mean, inds[1])
flatten_final = a
flatten_final=sparse.csr_matrix(flatten_final)

In [13]:
# steps = [(flatten_final)]
# from sklearn.pipeline import Pipeline
# pipeline = Pipeline(steps) # define the pipeline object.

In [14]:
X_test_t=[]
for i in X_test:
    X_test_t.append(FeatureExtration(i))

In [15]:
arr1 = [0,0,0,0,0,0]
for i in X_test_t:
    for k in range(500-len(i)):
        i.append(pd.Series(arr1))
arr2 = (np.array(X_test_t))
test_flaten=[]
for i in range(len(arr2)):
    test_flaten.append(flatten_list(arr2[i]))
    
test_flaten = np.asarray(test_flaten)

c = test_flaten
col_mean = np.nanmean(c, axis=0)
#Find indices that you need to replace
inds = np.where(np.isnan(c))
#Place column means in the indices. Align the arrays using take
c[inds] = np.take(col_mean, inds[1])
test_flaten = c
test_flaten = test_flaten.reshape(8, 3000)
test_flaten=sparse.csr_matrix(test_flaten)
z= pd.Series(y_test)
z.replace(r'(^ra.*)','ravibeligeri',regex=True, inplace = True)
z.replace(r'(^hr.*)','hrudayashiva',regex=True, inplace = True)
z.replace(r'(^som.*)','somashaker',regex=True, inplace = True)
y_test =z



# Model buidling

In [28]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
#model =SVC(kernel='linear')
#model = model.fit(flatten_final.toarray(), y_train)

#model = GaussianNB()
#model= model.fit(flatten_final.toarray(), y_train)

#model = RandomForestClassifier(max_depth=2, random_state=0)
#model = model.fit(flatten_final.toarray(), y_train)

model = DecisionTreeClassifier(random_state=0)
model = model.fit(flatten_final.toarray(), y_train)
print("Taining Accuracy :",model.score(flatten_final.toarray(), y_train))
print("Testing Accuracy: ",model.score(test_flaten.toarray(), y_test))


Taining Accuracy : 1.0
Testing Accuracy:  0.5


In [29]:
from sklearn.metrics import classification_report
predictions = model.predict(test_flaten.toarray())
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

hrudayashiva       0.50      0.33      0.40         3
ravibeligeri       0.50      0.50      0.50         2
  somashaker       0.50      0.67      0.57         3

 avg / total       0.50      0.50      0.49         8



In [ ]:
#---------------------------------------------------------------------------------------------
print("N-grams features- extracting ")
word_vector=TfidfVectorizer(analyzer="word", ngram_range=(2,3),max_features=3000, binary=False)
char_vector=TfidfVectorizer(analyzer="char", ngram_range=(2,3),max_features=3000, binary=False,min_df=0)
vectorizer=FeatureUnion([ ("chars",char_vector),("word",word_vector)] )
trainset=[]
classes=[]
n_gram_fvs_all={}
feature_names={}
for author,file in instance_by_author.items():
    trainset.append(file)
    matrix = vectorizer.fit_transform(trainset)
    n_gram_fvs_all[author]=matrix.toarray()
    feature_names[author]=vectorizer.get_feature_names()
    trainset.clear()
    classes.append(author)
    #print("matrix for the first document\n",matrix)
# for lable, vector in n_gram_fvs_all.items():
#     print(fileint("---------------n-grams features Extracted successfully-------------")
#------------------------------------------------------------------------------------------------------

In [ ]:
import nltk
%matplotlib inline
import seaborn as sns
# Compare the disputed papers to those written by everyone,
# including the shared ones.
authors = instance_by_author.keys()

# Transform the authors' corpora into lists of word tokens
federalist_by_author_tokens = {}
federalist_by_author_length_distributions = {}
for author in authors:
    tokens = nltk.word_tokenize(instance_by_author[author])
    # Filter out punctuation
    federalist_by_author_tokens[author] = ([token for token in tokens if any(c.isalpha() for c in token)])
    #print(federalist_by_author_tokens[author])
    # Get a distribution of token lengths
    token_lengths = [len(token) for token in federalist_by_author_tokens[author]]
    federalist_by_author_length_distributions[author] = nltk.FreqDist(token_lengths)
    federalist_by_author_length_distributions[author].plot(15,title=author)
    print(federalist_by_author_length_distributions[author], end='')